In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [7]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet").randomSplit(Array(0.1, 0.9))(0)

dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


lastException: Throwable = null


[label: int, tweet: string]

## Learning of model

In [8]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{HashingTF, Tokenizer, IndexToString, StringIndexer, VectorIndexer}

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")
val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")
val labelIndexer = new StringIndexer()
  .setInputCol("tweet")
  .setOutputCol("indexedLabel")
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(10)
val Array(trainingData, testData) = raw_sentiment.randomSplit(Array(0.7, 0.3))

val rf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(10)

val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
//   .setLabels(labelIndexer.labels)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, labelIndexer, featureIndexer, rf, labelConverter
                  ))

tokenizer = tok_4b3b043a4377
hashingTF = hashingTF_67123d0f9190
labelIndexer = strIdx_850e3c870c65
featureIndexer = vecIdx_bedcdf745067
trainingData = [label: int, tweet: string]
testData = [label: int, tweet: ...


[label: int, tweet: ...

In [10]:
val model = pipeline.fit(trainingData)

Exception in thread "Executor task launch worker for task 128" java.lang.SecurityException: Not allowed to invoke System.exit!
	at org.apache.toree.security.KernelSecurityManager.checkExit(KernelSecurityManager.scala:147)
	at java.lang.Runtime.halt(Runtime.java:273)
	at org.apache.spark.util.SparkUncaughtExceptionHandler.uncaughtException(SparkUncaughtExceptionHandler.scala:32)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:613)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)


lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 128, localhost, executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded

Driver stacktrace:
StackTrace: Driver stacktrace:
  at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1876)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:926)
  at org.a

In [ ]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-my_model")

In [8]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-my_model")

sameModel = pipeline_fc19f65b89cc


pipeline_fc19f65b89cc

In [9]:
val predictionsDF = sameModel.transform(inputDF)

predictionsDF = [tweet: string, words: array<string> ... 4 more fields]


[tweet: string, words: array<string> ... 4 more fields]

In [10]:
import org.apache.spark.sql.functions._
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [22]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.select(getProbability($"probability").alias("clean_probability"))
    .selectExpr("""
    CASE
        WHEN clean_probability > 0.55 THEN 'positive'
        WHEN clean_probability < 0.45 THEN 'negative'
        ELSE 'undefine'
    END AS clean_probability
    """)
    .withColumn("time_stamp", lit(current_timestamp()))
    .withWatermark("time_stamp", "20 seconds")
    .groupBy(window($"time_stamp", "10 seconds").alias("time_stamp"),
            $"clean_probability")
    .count()
    .select($"clean_probability", $"count")
    .show()
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6f7e466f

+-----------------+-----+
|clean_probability|count|
+-----------------+-----+
|         negative|   10|
|         positive|    5|
+-----------------+-----+

+-----------------+-----+
|clean_probability|count|
+-----------------+-----+
|         undefine|    4|
|         negative|    9|
|         positive|    3|
+-----------------+-----+

+-----------------+-----+
|clean_probability|count|
+-----------------+-----+
|         undefine|    5|
|         negative|    7|
|         positive|    4|
+-----------------+-----+

+-----------------+-----+
|clean_probability|count|
+-----------------+-----+
|         negative|    8|
|         positive|    8|
|         undefine|    4|
+-----------------+-----+

+-----------------+-----+
|clean_probability|count|
+-----------------+-----+
|         negative|    5|
|         positive|    8|
|         undefine|    2|
+-----------------+-----+

+-----------------+-----+
|clean_probability|count|
+-----------------+-----+
|         negative|    5|
|      